In [1]:
import urllib.request
import re
import pandas as pd
from bs4 import BeautifulSoup
import time
import requests
from timeit import default_timer as timer
from os.path import expanduser as ospath



def web_scrap(pat_num):
    
    start = timer()
    
    claim_list, abstr_list, title_list, publication_num, family_id, application_num, application_num_orig,  =([]for i in range(7))
    
    count = 0
    
    for pat in pat_num:
        
        pat = str(int(pat))
        
        url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=1&p=1&f=G&l=50&d=PTXT&S1='+pat+'.PN.&OS=pn/'+pat+'&RS=PN/'+pat+''
        headers = {'connection':'close','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
        
        time.sleep(10)
        
        req = urllib.request.Request(url,headers = headers)
        resp = urllib.request.urlopen(req)
        respData = resp.read()
        
        tit = re.findall(r"<FONT size\=\"\+1\">(.*?)</FONT>",str(respData))#title of the patent
        claim = re.findall(r'<CENTER><b><i>Claims</b></i></CENTER> <HR> <BR><BR>(.*?)<HR>',str(respData))
        publn_nr = re.findall(r'<TITLE>United States Patent: ([0-9]+)</TITLE>',str(respData))#Title num of the patent
        fam_id = re.findall(r'Family ID.*?(\d{5,9}).*?Appl. No.', str(respData))#Family ID
        appl_num = re.findall(r'Appl. No.:.*?<b>(.*?)</b>', str(respData))#Appl. No. ex. 08/544,281
        appl_num_orig = re.findall(r'<U>Publication Date.*?(US\s+[0-9]+\s+[A-Z][0-9])', str(respData))#ex. 'US 20080246285 A1'
        soup = BeautifulSoup(respData,'html.parser')
        abst = str(soup.find('p'))#find the first paragraph which is the abstract
        
        #append the string to each lists
        if len(publn_nr) == 0:
            publn_nrs = ''
        else:
            publn =  list(publn_nr)#convert the string found in a list of chracters
            publn_nrs = ''.join(publn)#join back in a string to get rid of the squared brackets
        publication_num.append(publn_nrs)
        
        if len(fam_id) == 0:
            fam_ids = 'NA'
        else:
            fam_ids =  fam_id
        family_id.append(fam_ids)
        
        if len(appl_num) == 0:
            appl_nums = ''
        else:
            appl_nums =  re.sub(r'[[\' /, \]]',"",str(appl_num)).strip()#ex. from [' 08/544,281'] to ['08544281']
        application_num.append(appl_nums)
        
        if len(appl_num_orig) == 0:
            appl_num_origs = ''
        else:
            appl_num_or =  list(appl_num_orig)#see for publn_nr
            appl_num_origs = ''.join(appl_num_or)    
        application_num_orig.append(appl_num_origs)
        
        if len(claim)==0:
            claims = 'NA'
        else:
            claims = claim
        claim_list.append(claims)
        
        if abst is None:
            absts = 'NA'
        else:
            absts = abst
        abstr_list.append(absts)
        
        if len(tit)==0:
            titl = 'NA'
        else:
            titl = tit
        title_list.append(titl)
        
        count +=1
        
        print(count, end=" ")
        

    #to clean the strings
    for items in range(len(title_list)):   
        title_list[items] = str(title_list[items]).replace("\\n", "")
        title_list[items] = str(title_list[items]).replace("\\", "")
        title_list[items] = title_list[items][1:-1] 
        abstr_list[items] = abstr_list[items].replace("    ", "")
        abstr_list[items] = abstr_list[items].replace("\n", "")
        abstr_list[items] = abstr_list[items].replace("<p>", "")
        abstr_list[items] = abstr_list[items].replace("</p>", "")
        abstr_list[items] = re.sub(r'^ ',"",str(abstr_list[items])).strip()
        abstr_list[items] = subscript(abstr_list[items])
        claim_list[items] = re.sub(r'<BR>',"",str(claim_list[items])).strip() 
        claim_list[items] = claim_list[items].replace("\\n", "")
        claim_list[items] = claim_list[items].replace("\\", " ")
        claim_list[items] = claim_list[items][1:-1]
        
        
    claim_list = split_claims(claim_list)
    
    end = timer()
    
    print('web_scrap',end - start)
    
    #reference - adapted from - https://github.com/imoisharma/U.S.-Patents-Claims/blob/master/U.S.%20Patents.ipynb
    
    return claim_list, abstr_list, title_list, publication_num, family_id, application_num, application_num_orig



def split_claims(ls_claim):
    
    """splits the claims
    @ls_claim = list of claims"""
    
    start = timer()
    claim_sep = []
    
    for j in range(len(ls_claim)):
        if ls_claim[j] != 'NA':
            claim = []
            claim_minus_1 = re.findall(r'\.(\s+2.*)',str(ls_claim[j]))#everything from claim 2
            claim_counter = re.findall(r'\.\s+([0-9]+)\.',str(ls_claim[j]))#numeric list from the second cl to the end
            claim_one = re.findall(r'(^.*?.)\s+2\. ',str(ls_claim[j]))#first claim
            
            claim.extend(claim_one)
            
            count = len(claim_counter)
            
            for i in range(len(claim_counter)):
                if count >= 2:
                    if i ==0:
                        regular =  '[\"\']\s*(2\.\s+.*?[a-zA-Z]+.*?)\s*3\.'#for the claim 2
                        next_claim = re.findall(regular,str(claim_minus_1))
                        claim.extend(next_claim)
                    else:
                        regular =  '\.\s*( '+ claim_counter[i]+'\.\s+.*?[a-zA-Z]+.*?)\s*'+claim_counter[i+1]+'\.'
                        next_claim = re.findall(regular,str(claim_minus_1))
                        claim.extend(next_claim)
                else:
                    regular =  '.\s+('+claim_counter[i]+'\. .*?[a-zA-Z]+.*?.*)]'#for the last claim
                    next_claim = re.findall(regular,str(claim_minus_1))
                    claim.extend(next_claim)
                count -= 1       
        else:
            continue
        claim_sep.append(claim)
    end = timer()
    print('split_claims',end - start)
    return claim_sep 


def build_dataset(*args):
    
    """built a database in which the abstract and
    each claim has a separate line
    
    @*arg = it allows to add more columns in the database"""
    
    start = timer()
    
    #list of patents with abstract and claims text in separate rows
    list_pat = []
    
    for i in range(len(claim_list)):
        
        docdb = data['docdb_family_id'].iloc[i] # family ID
        publn_nr_patstat = data['publn_nr'].iloc[i] # publn num
        appln_id = data.loc[data['docdb_family_id'] == docdb].iloc[[0,]].index[0]
        
        #set the first row of the patents with only the text for the abstract
        temp = [[ appln_id, publn_nr_patstat , docdb , publication_num[i], application_num[i],application_num_orig[i], title_list[i][1:-1], 'Abstract' ,abstr_list[i]]]
        
        #temporary rows to be added 
        temp_2 = []
        
        for j in range(len(claim_list[i])):
            
            #set a new rows for each claims in a patent 
            new_ls = [appln_nr ,publn_nr_patstat , docdb, publication_num[i], application_num[i] , application_num_orig[i], title_list[i][1:-1]]
            claim = claim_list[i][j]
            
            if j == 0:
                #clean the first claim
                claim = re.sub(r'(.*?[\:\.]\s*'+str(j+1)+'\.\s*) ',"",str(claim)).strip() 
                b = '[\']'
                claim = claim.replace(b, "")
                new_ls.extend(['Claim ' + str(j+1), str(claim)])
            
            else:
                #clean all the rest of the claims
                claim=re.sub(r'('+ str(j+1)+'\.\s+)',"",str(claim)).strip()
                claim=re.sub(r'([\"\"])|(\\)|([\'\'])|(\[\]) ',"",str(claim)).strip()
                claim=re.sub(r'\'',"",str(claim)).strip()
                new_ls.extend(['Claim ' + str(j+1), str(claim)])
            
            temp_2.append(new_ls)
        
        temp.extend(temp_2)
        
        list_pat.extend(temp)
    end = timer()
    
    print('build_dataset',end - start)
    
    return pd.DataFrame(list_pat, columns = ['Appln_ID', 'Publn_nr_PASTAT','Docdb_Family_ID', 'Publn_Nr', 'Appln_No', 'Appln_Num_Orig','Title', 'Type', 'Text'])


## Web scraped the patents in small batches 

In [2]:
data = pd.read_excel(ospath('~/final_project/A_Data_collection/USPTO/PATSTAT_USPTO_patents/tb201LC_Query.xlsx'), index_col=0)
data = data.sort_values(by='docdb_family_id',  ascending=[False])#sorted by family ids (descending order)
data = data[0:500]


pat_num = data['publn_nr']



claim_list, abstr_list, title_list, publication_num, family_id, application_num, application_num_orig = web_scrap(pat_num)
dataset = build_dataset(title_list, claim_list, abstr_list, application_num_orig, publication_num)

dataset.to_excel(ospath('~/final_project/A_Data_collection/USPTO/Data_collected/db_LC_0_499.xlsx'), index = None, header = True)

c:\users\rober\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:53: FutureWarning: Possible nested set at position 1


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [3]:
dataset.head()

,Appln_ID,Publn_nr_PASTAT,Docdb_Family_ID,Publn_Nr,Appln_No,Appln_Num_Orig,Title,Type,Text
0,353625209,10085792,67148336,10085792.0,13270701.0,US 20120116260 A1,Surgical instrument with motorized attachment ...,Abstract,An apparatus includes an instrument body and t...
1,353625209,10085792,67148336,10085792.0,13270701.0,US 20120116260 A1,Surgical instrument with motorized attachment ...,Claim 1,An apparatus comprising: (a) an instrument bod...
2,353625209,10085792,67148336,10085792.0,13270701.0,US 20120116260 A1,Surgical instrument with motorized attachment ...,Claim 2,"The apparatus of claim 1, wherein the switch i..."
3,353625209,10085792,67148336,10085792.0,13270701.0,US 20120116260 A1,Surgical instrument with motorized attachment ...,Claim 3,"The apparatus of claim 2, wherein the switch i..."
4,353625209,10085792,67148336,10085792.0,13270701.0,US 20120116260 A1,Surgical instrument with motorized attachment ...,Claim 4,"The apparatus of claim 3, wherein the motor is..."
